# Workshop i nätverksanalys och webbskrapning
I dagens workshop ska vi kombinera veckans moment i nätverk och webbskrapning till att titta på data från Twitter. Vi kommer *inte* att använda oss av deras API, eftersom det har så många begränsningar. Istället kommer vi använda ett inofficiellt verktyg i python som faktiskt skrapar datan från webben.

In [1]:
from course.riksdagsdata import Riksdagsdata
from course.riksdagsdata import color_from_party
from course.data import color_from_betweenness_centrality, color_from_degree_centrality
from course.riksdagsdata import nodes_and_edges_from_motions

import course.network
import networkx as nx
from bokeh.io import show

Loading BokehJS ...

## Innehåll
### Riksdagens API
Börja med att ange *endpointen* för API:t hos Riksdagens Öppna Data. Vi skapar då en anslutning direkt till deras databas.

In [2]:
API = 'http://data.riksdagen.se/dokumentlista/'
riksdag = Riksdagsdata(API)

Förbered därefter en sökfråga på motioner. Ni kan välja dels en sökterm ``query``, samt intervall för när motionen presenterades, genom ``start_date-end_date``. Begränsa lämpligtvis hur många träffar ni får med ``limit``.

In [16]:
# Skapa ett API-anrop för en sökfråga
query = 'skatt'
start_date = '2014-11-01'
end_date = '2018-11-01'

motion_generator = riksdag.motions(query, 
                                   start_date=start_date, 
                                   end_date=end_date, 
                                   sort='rel', 
                                   limit=1000)

# Hämta motioner i en lista
motions = [m for m in motion_generator]

print(f"Antalet motioner är: {len(motions)}")

Antalet motioner är: 1000


Motionernas metadata är nu nedladdad och ni kan inspektera dem genom att printa, ifall ni är nyfikna:

In [17]:
for motion in motions:
    print(f"ID: {motion.id}, {motion}")

ID: H4023356, Motion: Skatt på pension ska vara på samma nivå som skatt på förvärvsinkomst
ID: H602173, Motion: Skatter och avgifter på lönebeskedet
ID: H5022985, Motion: Skatt och transparens för löntagare
ID: H5022521, Motion: Utgiftsområde 3 Skatt, tull och exekution
ID: H5023840, Motion: Utgiftsområde 3 Skatt, tull och exekution
ID: H5023722, Motion: Synliggörande av och information om skatter
ID: H5023319, Motion: Minskat skatteslöseri och tydlig information om skatter
ID: H5023774, Motion: Skatter
ID: H5023693, Motion: Utgiftsområde 3 Skatt, tull och exekution
ID: H5023154, Motion: Öka kunskaperna om skatterna
ID: H5022451, Motion: Skatt på biodrivmedel
ID: H5022590, Motion: Synliggörande av osynliga skatter
ID: H5023218, Motion: Avskaffad skatt på RUT-tjänster för äldre
ID: H5023786, Motion: Utgiftsområde 3 Skatt, tull och exekution
ID: H5022737, Motion: Synliggörandet av dolda skatter
ID: H5022325, Motion: Differentierade skatter för lokalproducerat
ID: H5023572, Motion: Din rä

Om ni vill läsa en enskild motion kan ni ange dess *id* (genom ``motion.id``) i webbläsaren:

```
http://data.riksdagen.se/dokument/<id>
```

genom att substituera ``<id>`` med ert motions-id. Exempel: http://data.riksdagen.se/dokument/H602213

### Nätverk

Vi kan nu visualisera motionernas författare som ett nätverk. Alla motioner har ett antal *motionärer* som till hör ett eller flera partier. Det kan därför vara av intresse att studera samarbetsbenägenheten mellan olika partier och individer. 

Varje nod kommer i detta fall att utgöras av en motionär och riksdagsledamot, och kanter mellan noder indikerar att de har författat minst en motion tillsammans.

In [18]:
# Konstruera noder och kanter på samma format som i tidigare föreläsning
V, E = nodes_and_edges_from_motions(motions)

# Skapa ett nätverk.
G = course.network.Network(V, E)

Vi vill också tillföra extra information till nätverket via färgläggning. Vi kan dels ge varje nod en färg efter partiet som motionären tillhör, alternativt efter någon form av centralitetsmått. Avkommentera nedan genom att ta bort grinden ``#`` och byt ut mot den färgsättning ni tycker är lämplig.

In [19]:
colors = color_from_party(G)
# colors = color_from_betweenness_centrality(G)
# colors = color_from_closeness_centrality(G)
# colors = color_from_degree_centrality(G)

In [26]:
g = G.plot(nx.spring_layout, 
       title=f'Motionärer i Sveriges riksdag om {query} ({start_date} till {end_date})',
       fig_size=(500, 500),
       fill_color=colors,
       k=0.3,
       scale=0.5, 
       center=(0.5, 0.5)
    )

show(g)

## Uppgifter
Workshopens uppgift går ut på att utforska riksdagens API och generera insikter via nätverksanalys.

Det finns ett antal parametrar ni kan variera:
- Sökterm: Undersök ett par olika eller närbesläktade ämnen och se hur motionssamarbetena varierar.
- Tidsperiod: Undersök förändringen över tid hos ett ämne och studera ifall motionssamarbetena ändras.
- Storleken på nätverket: Sänk övre gränsen på antalet motioner ni vill hämta. Är nätverket fortfarande representativt med färre noder, eller ändras strukturen avsevärt med fler motioner?

Ni skall också studera nätverkens struktur:
- Undersök ifall det bildas kluster av partier och vilka som samarbetar med vilka.
- Finns det några individer som agerar brygga mellan olika kluster eller partier?
- Studera centraliteten hos olika individer i ert nätverk - vad säger de olika måtten om individerna?

Frågorna skall besvaras och sammanställas i en kortare presentation i slutet på workshopen. Spara gärna bilder till denna presentation eller var förberedda med att variera parametrarna i notebooken.

In [ ]:
'https://data.riksdagen.se/dokumentlista/?sok=&doktyp=mot&utformat=iddump&a=s'